# Code Review

## Object


## Smelly example #1

```ts
function dayOfYear(month: number, dayOfMonth: number, year: number): number {
    if (month === 2) {
        dayOfMonth += 31;
    } else if (month === 3) {
        dayOfMonth += 59;
    } else if (month === 4) {
        dayOfMonth += 90;
    } else if (month === 5) {
        dayOfMonth += 31 + 28 + 31 + 30;
    } else if (month === 6) {
        dayOfMonth += 31 + 28 + 31 + 30 + 31;
    } else if (month === 7) {
        dayOfMonth += 31 + 28 + 31 + 30 + 31 + 30;
        dayOfMonth += 31 + 28 + 31 + 30 + 31 + 30 + 31;
    } else if (month === 9) {
        dayOfMonth += 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31;
    } else if (month === 10) {
        dayOfMonth += 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30;
    } else if (month === 11) {
        dayOfMonth += 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30 + 31;
    } else if (month === 12) {
        dayOfMonth += 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30 + 31 + 31;
    }
    return dayOfMonth;
}

```
## Don't repeat yourself(DRY)

两个地方有相同或者相似的代码，会存在安全隐患——一个地方修复了，另一个地方没有修复。
**Copy-and-paste is an enormously tempting programming tool, and you should feel a frisson of danger run down your spine every time you use it.**

### Reading exercise
代码中的另一种重复是dayOfMonth+=。假设您有一个列表： const monthLength: Array<number> = [undefined, 31, 28, 31, 30, ..., 31]; 这monthLength[month]就是 中的天数month，其中一月用 1 表示。

以下哪一个代码框架可用于充分干燥代码，使其dayOfMonth+=仅出现一次?

- [x] for (let m = 1; m < month; ++m) { ... }
- [ ] switch (month) { case 1: ...; break; case 2: ...; break; ... }
- [x] while (m < month) { ...; m += 1; }
- [x] if (month === 1) { ... } else { ... dayOfYear(month-1, monthLength[month-1], year) ... }

## Comment where needed
specification是一种注释（comment），出现在函数和类之上，记录这个函数和类的行为。
```ts
/**
 * Compute the hailstone sequence.
 * See https://en.wikipedia.org/wiki/Collatz_conjecture#Statement_of_the_problem
 * @param n starting number of sequence; requires integer n > 0.
 * @returns the hailstone sequence starting at n and ending with 1.
 *         For example, hailstone(3) = [3, 10, 5, 16, 8, 4, 2, 1].
 */
function hailstoneSequence(n: number): Array<number> {
    ...
}
```
通过copy的手段引用或者改造的代码
```ts
// reverse a string of digits
// see https://stackoverflow.com/questions/1611427/reversing-a-string-in-javascript
const digitsReversed = digits.split('').reverse().join('');
```
之所以这么做，一是为了避免违反开源协议，二是为了防止代码过时。

有些注释是没有必要的，例如，直接将代码翻译成英语并不能提高理解能力，应该假设读者至少了解 TypeScript：

```Ts
while (n !== 1) { // test whether n is 1   (don't write comments like this!)
   ++i; // increment i
   l.push(n); // add n to l
}
```
但晦涩的代码应该得到注释：
```Ts
const sum = n*(n+1)/2;  // Gauss's formula for the sum of 1...n

// here we're using the sin x ~= x approximation, which works for very small x
const moonDiameterInMeters = moonDistanceInMeters * apparentAngleInRadians;
```

通常最好将代码按照目的分成段落，行组，以注释开始作为每一个段落的主题，解释目的：
```Ts
// prepare the rocket
...
...
...

// launch the rocket
...
...
...

// enter orbit
...
...
```
一旦你开始以这种方式思考你的代码，你可能会意识到这些段落中的部分或全部都应该是辅助函数，然后注释可能自然而然地变成命名良好的函数调用：
```Ts
prepareRocket(...);
launchRocket(...);
enterOrbit(...);
```

### Reading Exercise
哪些注释是对代码有用的补充？独立考虑每条评论，就好像其他评论不存在一样。
```ts
/**
 * @param month month of the year, where January=1 and December=12  [C1]
 */
function dayOfYear(month: number, dayOfMonth: number, year: number): number {
    if (month === 2) {     // we're in February  [C2]
        dayOfMonth += 31;  // add in the days of January that already passed  [C3]
    } else if (month === 3) {
        dayOfMonth += 59;  // month is 3 here  [C4]
    } else if (month === 4) {
        dayOfMonth += 90;
    }
    ...
    } else if (month === 12) {
        dayOfMonth += 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30 + 31 + 31;
    }
    return dayOfMonth; // the answer  [C5]
}
```
[C1]是常规做法，也是specification的要求做法，解释了参数month的意义，此路例并不完整；
[C2]和[C3]分别解释数字2,和31的意义，这个有助于读者的理解；
[C4]试图解释3的意义，但是对比[C2],他就显得十分粗糙；



## Fail fast
defination：尽可能早地暴露bug，越早暴露，越容易修复。

### Reading Exercise
以下是对dayOfYear()的改进，是否会失败的更快？

```Ts
function dayOfYear(month: number, dayOfMonth: number, year: number): number {
    if (month < 1 || month > 12) {
        return -1;
    }
    ...
}
```

```Ts
function dayOfYear(month: number, dayOfMonth: number, year: number): number {
    if (month < 1 || month > 12) {
        throw new Error("month out of range");
    }
    ...
}
```
这两种设计很相似，但是第二种更好。因为第一种尽管传递一个特殊值，但它并不会立即抛出错误，而是产生错误答案。这导致之后需要亲自排查。

```ts
enum Month { JANUARY, FEBRUARY, MARCH, ..., DECEMBER };
function dayOfYear(month: Month, dayOfMonth: number, year: number): number {
    ...
}
```
这类似于要求月份为字符串，因为它会导致先传递日期或年份的可能错误产生静态错误。

但它甚至比使用字符串更好，因为调用者可以传递的唯一值是十二个月之一。任何其他值都会被编译器静态拒绝。相比之下，使用字符串，调用者可以传递拼写错误的月份名称（如“Aprile”），或不同的语言（如“январь”），或缩写如“Jan”（代码尚未准备好），并且防止此函数可能会出现动态错误。使用固定的枚举值会在编译时捕获更多错误。


```TS
function dayOfYear(month: number, dayOfMonth: number, year: number): number {
    if (month === 1) {
        ...
    } else if (month === 2) {
        ...
    }
    ...
    } else if (month === 12) {
        ...
    } else {
        throw new Error("month out of range");
    }
}
```

此函数检查了月份范围是否在 1-12 之间，如果不是，则失败速度更快，并出现动态错误。它展示了一种良好的防御性编程实践：用 else 子句结束每个多情况 if 语句（或 switch 语句），该子句有效地表示“如果不是任何这些情况，那么一定存在错误。

但即使这种方法失败得更快，从 DRY 的角度来看，它仍然是一个糟糕的解决方案。


## 避免神奇数字(magic number)
有一个计算机科学的笑话，说计算机科学家唯一能理解的数字是0、1，有时是2。

所有其他常数都被称为魔法常数，因为它们就像凭空出现一样，没有任何解释。

解释数字的一种方法是使用注释，但更好的方法是将数字声明为一个命名常量，并为其命名一个清晰明确的名称。

dayOfYear 充满了神奇的数字，这说明了应该避免使用数字的原因：

* 数字的可读性比名称差。在dayOfYear中，月份值2, ..., 12的可读性要比FEBRUARY, ..., DECEMBER好得多。

* 常量将来可能需要更改。使用一个命名的常量，而不是在不同的地方重复字面数字，更能适应变化。

* 常数可能依赖于其他常数。在dayOfYear中，神秘的数字59和90就是特别有害的例子。它们不仅没有注释和文档，而且实际上是程序员手工计算的结果--将某些月份的长度相加。这不容易理解，也不便于更改，更不安全。不要对手工计算的数字进行硬编码。使用一个命名常量，明显地计算出其他命名常量的关系。

那么那些看似恒定永恒的常数呢，比如π，或者三角形的总角，或者引力常数G？是否有必要为不依赖于其他常数的数学和物理学基本常数命名？首先，数字本身的表达可能比较复杂，因此多次重复并不能避免出现错误。是3.14159265358979323846还是3.1415926538979323846？其次，即使这些数字也包含了将来可能改变的设计决策，例如精度的位数或测量单位。当这些设计决定发生变化时，一个命名常数更能适应变化。

如果您的代码中出现了大量的神奇数字，这可能表明您需要退一步，将这些数字视为数据而不是命名常量，并

```Ts
/** pluralize `word` using the pluralization rules of `language` */
function pluralize(word: string, language: number) { ... }

pluralize('dog', 8);  // returns 'dogs' in English
pluralize('Hund', 7); // returns 'Hunde' in German
```
更改后,
如果语言集在编译时是固定的，那么使用枚举会更好
```Ts
enum Language { English, Deutsch }
/** pluralize `word` using the pluralization rules of `language` */
function pluralize(word: string, language: Language) { ... }

pluralize('dog', Language.English);  // returns 'dogs'
pluralize('Hund', Language.Deutsch); // returns 'Hunde'
```
但是，如果语言集需要在运行时更改（可能是通过从文件加载特定于语言的规则），那么您可能更喜欢使用 const string 通用的设计：

```Ts
/** pluralize `word` using the pluralization rules of `language` */
function pluralize(word: string, language: string) { ... }

const ENGLISH = 'English';
const DEUTSCH = 'Deutsch';

pluralize('dog', ENGLISH);  // returns 'dogs'
pluralize('Hund', DEUTSCH); // returns 'Hunde'
```

动态使用“string”，静态用枚举。


## 一个变量，一个用途
在dayOfYear示例中，参数dayOfMonth被重复用于计算一个完全不同的值--函数的返回值，而这个返回值并不是月份的日期。

不要重复使用参数，也不要重复使用变量。变量并不是编程中的稀缺资源。自由引入变量，给它们起个好名字，当您不再需要它们时就停止使用。如果一个变量在几行之后突然有了不同的含义，就会使读者感到困惑。

这不仅是一个易于理解的问题，也是一个防止错误和随时准备更改的问题。

尤其是函数参数，通常不应修改。（这对于随时准备更改是很重要的--在将来，函数的其他部分可能想知道函数的原始参数是什么，所以在计算时不应该把它们删除）。对尽可能多的变量使用const是个好主意，因为const关键字表示变量不应该被重新分配，TypeScript编译器会静态检查它。不幸的是，TypeScript还不支持将函数参数声明为const，以防止它在函数体中被重新赋值。

## Smelly example #2
在dayOfYear的例子中有一个潜在的bug，它根本不处理闰年。作为解决这个问题的一部分，假设我们编写一个闰年函数。
```Ts
function leap(y:number):boolean {
    let tmp=y.toString();
    if(tmp[2]==='1'||tmp[2]==='3'||tmp[2]==='5'||tmp[2]==='7'||tmp[2]==='9') {
        if(tmp[3]==='2'||tmp[3]==='6') return true; /*R1*/
        else
            return false; /*R2*/
    }else{
        if(tmp[2]==='0' && tmp[3]==='0') {
            return false; /*R3*/
        }
        if(tmp[3]==='0'||tmp[3]==='4'||tmp[3]==='8') return true; /*R4*/
    }
    return false; /*R5*/
}
```
这段代码中隐藏着哪些bug？
1. 使用了magic number，
2. 没有在函数内部加上注释
3. 


我们已经讨论过哪些风格问题？

## 使用好的名字
好的函数和变量名称应该很长并且具有自我描述性。通常可以通过使代码本身更具可读性并使用更好的名称来描述函数和变量来完全避免注释。
例如，您可以重写
```ts
const tmp = 86400;  // tmp is the number of seconds in a day (don't do this!)
```
作为：
```ts
const secondsPerDay = 86400;
```
遵循语言的词汇命名约定。在 Python 和 TypeScript 语言中，类通常以大写字母开头，变量名和函数名以小写字母开头。但这两种语言的不同之处在于如何将多词短语呈现为函数或变量名称。Python 使用蛇形命名法（下划线分隔短语中的单词），而 TypeScript 使用驼峰命名法（第一个单词后的每个单词大写，如startsWithor中getFirstName）。

类 Java 语言也使用大写来区分全局常量（const任何函数外部的声明）与变量和局部常量。 ALL_CAPS_WITH_UNDERSCORES用于全局常量。但是在函数内部声明的局部变量（包括secondsPerDay上面的局部常量）使用camelCaseNames.

在任何语言中，函数名称通常是动词短语，例如getDate或isUpperCase，而变量和类名称通常是名词短语。选择简短的单词，并且要简洁，但避免缩写。例如，message比 更清晰msg，并且word比 好得多wd。请记住，课堂上和现实世界中的许多队友都不是以英语为母语的人，对于非母语人士来说缩写可能更难。

[有效代码命名](https://medium.com/@rabinovichsagi/effectively-naming-software-thingies-fcea9d78a699)
[Hungarian notation](https://en.wikipedia.org/wiki/Hungarian_notation)
推荐书籍：clean code


## 使用空格和标点符号来帮助读者
在代码行中添加空格以使其易于阅读。跳跃的例子有很多行，它们挤在一起，难以阅读：
```ts

    if(tmp[2]==='1'||tmp[2]==='3'||tmp[2]==='5'||tmp[2]==='7'||tmp[2]==='9') {
```
===在二元运算符（如和 ）周围使用空格||来将它们分开并使它们更加明显，并在多行上使用对齐方式以使相似点和差异向读者突出：
```ts
    if (   tmp[2] === '1'
        || tmp[2] === '3'
        || tmp[2] === '5'
        || tmp[2] === '7'
        || tmp[2] === '9' ) {
```

* [为什么最好不要省略{}](https://stackoverflow.com/questions/2125066/is-it-a-bad-practice-to-use-an-if-statement-without-curly-braces)

* [最好加上；在TS中](https://stackoverflow.com/questions/444080/do-you-recommend-using-semicolons-after-every-statement-in-javascript)

## smelly example #3

```ts
let LONG_WORD_LENGTH = 5;
let longestWord;

function countLongWords(text: string): void {
    let words: Array<string> = text.split(' ');
    if (words.length === 0) {
        console.log("0");
        return;
    }
    let n = 0;
    longestWord = "";
    for (let word of words) {
        if (word.length > LONG_WORD_LENGTH) ++n;
        if (word.length > longestWord.length) longestWord = word;
    }
    console.log(n);
}

```

## 不要使用全局变量
```ts
let LONG_WORD_LENGTH = 5;
let longestWord;

function countLongWords(text: string): void {
    let words: Array<string> = text.split(' ');
    if (words.length === 0) {
        console.log("0");
        return;
    }
    let n = 0;
    longestWord = "";
    for (let word of words) {
        if (word.length > LONG_WORD_LENGTH) ++n;
        if (word.length > longestWord.length) longestWord = word;
    }
    console.log(n);
}
```
避免使用全局变量。让我们来解释一下全局变量的含义。全局变量是

变量名：其值可以更改的变量
全局变量：在程序的任何地方都可以访问和更改。
Global Variables Are Bad（全局变量是坏的）很好地列举了全局变量的危险。

在TypeScript中，全局变量是在函数定义之外使用let或var声明的。

然而，如果使用const来声明，并且变量的类型是不可变的，那么变量名就变成了全局常量。全局常量可以在任何地方被读取，但永远不会被重新分配或变异，因此风险消失了。全局常量很常见，也很有用。

一般来说，将全局变量转换为参数和返回值，或者将它们放在调用函数的对象中。在以后的阅读中，我们会看到很多这样做的技巧。

[全局变量是很糟糕的](http://wiki.c2.com/?GlobalVariablesAreBad)

### Kinds of variables in snapshot diagrams

## 函数应该返回结果，而不是打印结果

countLongWords 还没有准备好进行更改。它使用 console.log() 将部分结果发送到控制台。这意味着，如果您想在其他上下文中使用它--数字需要用于其他目的，例如计算而不是人眼--就必须重写它。

一般来说，只有程序的最高层部分才应该与人类用户或控制台交互。低级部分应该将输入作为参数，将输出作为结果返回。唯一的例外是调试输出，它当然可以打印到控制台。但是这种输出不应该是设计的一部分，而应该是调试设计的一部分。


### reading exercise

该countLongWords函数实际上有两个结果——n当前打印到控制台，longestWord当前存储在全局变量中。以下哪个函数签名能够返回两个结果，而无需打印到控制台或使用全局变量？在给定的选项中，哪一个是最佳选择？

能够返回两个结果：

- [ ] function countLongWords(text: string): number
- [x] function countLongWords(text: string): string
- [x] function countLongWords(text: string): Array<string>
- [ ] function countLongWords(text: string): void
- [x] function countLongWords(text: string): [string, number]
- [x] function countLongWords(text: string): { count: number, longestWord: string}
返回两个结果的最佳选择：

- [ ] function countLongWords(text: string): void
- [ ] function countLongWords(text: string): number
- [ ] function countLongWords(text: string): string
- [ ] function countLongWords(text: string): Array<string>
- [ ] function countLongWords(text: string): [string, number]
- [x] function countLongWords(text: string): { count: number, longestWord: string }


string可以想象，可以为此使用返回值，例如通过返回n + "," + longestWord。但这是一个糟糕的选择，因为它无法从静态检查中受益。函数的调用者可能会意外地使用返回值，就好像它只是一样longestWord（没有首先拆分并删除该n值），并且编译器不会告诉调用者他们做错了什么。

二元素Array<string>稍微好一些，其中一个元素为n（转换为 a string），另一个元素为longestWord。但这仍然具有较弱的静态检查，因为调用者可能会混淆它们所处的顺序，而编译器不会出错。

像这样的元组[string, number]会好一点。它指定返回值必须是双元素数组，其中第一个元素是字符串，第二个元素是数字。

但对象类型 { count: number, longestWord: string }将是静态检查的最佳选择。返回值的两部分有明确的命名和明确的类型，一部分是数字，另一部分是字符串。使用此类型声明，该函数将返回一个对象文字，如下所示：

return { count: n, longestWord: longestWord };
调用者可以使用解构赋值将返回值的两部分提取到自己的局部变量中：

const { count, longestWord } = countLongWords(text);

## 避免特殊情况的代码

```ts

if (words.length === 0) {
    console.log("0");
    return;
}
let n = 0;
longestWord = "";
for (let word of words) {
    if (word.length > LONG_WORD_LENGTH) ++n;
    if (word.length > longestWord.length) longestWord = word;
}
console.log(n);

```
第一个if的statement实际上是不需要的，因为当长度为0时，for statement不会作任何事情，对于0同样也会打印出来。

### Reading exercise

Smelly Example #3 (reproduced below) starts with an if statement that handles a special case, prints 0, and returns immediately. But this makes the behavior of the special case inconsistent with other cases where the code would print 0, for example when the list is nonempty but has no long words in it. Which line of code from the general-case code did the special case also need to do?
```ts


         if (words.length === 0) {
             console.log("0");
             return;
         }
/*A*/    let n = 0;
/*B*/    longestWord = "";
/*C*/    for (let word of words) {
/*D*/        if (word.length > LONG_WORD_LENGTH) ++n;
/*E*/        if (word.length > longestWord.length) longestWord = word;
         }
/*F*/    console.log(n);
```
- [ ] A
- [ ] B
- [ ] C
- [ ] D
- [x] E
- [ ] F
The special case code failed to clear the global variable longestWord, which does get cleared in other cases where the result is 0. This odd variation in behavior is likely to lead to unexpected bugs in code depending on this function.

Note that the best way to fix this is not to copy longestWord = "" into the special-case if statement. Instead the special case should be deleted entirely.

## summary

代码审查是一种广泛使用的通过人工检查来提高软件质量的技术。代码审查可以发现代码中存在的多种问题，但作为入门，本读物讨论了良好代码的一般实践：

* 不要重复 (DRY)
* 在需要的地方进行注释
* 快速失败
* 避免神奇数字
* 每个变量只有一个目的
* 使用好的名称
* 使用空白和标点符号帮助读者阅读
* 不要使用全局变量
* 函数应返回结果，而不是打印结果
* 避免使用特例代码
今天阅读的主题与良好软件的三个关键属性相关联，具体如下：

远离错误。一般来说，代码审查是由人工审查员来发现错误的。DRY代码可以让您只在一个地方修复错误，而不必担心错误会传播到其他地方。用清晰的注释来记录您的假设，可以降低其他程序员引入错误的可能性。快速失败原则可以尽早发现错误。避免使用全局变量可以更容易地定位与变量值相关的错误，因为非全局变量只能在代码中有限的地方进行更改。

易于理解。代码审查是发现晦涩或混乱代码的唯一途径，因为其他人也在阅读并试图理解这些代码。使用明智的注释、避免神奇的数字、为每个变量保留一个目的、使用好的名称以及良好地使用空白都可以提高代码的可理解性。

为变更做好准备。当代码审查由经验丰富的软件开发人员完成时，代码审查会有所帮助，他们可以预测可能发生的变化，并提出防范措施。DRY代码更易于更改，因为只需在一处进行更改。返回结果而不是打印结果使代码更容易适应新的用途。

代码审查并不是软件开发实践中唯一一个证据确凿的做法。另一个实践是：[睡眠](https://increment.com/teams/the-epistemology-of-software-quality/)
